In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train =pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
data = train.append(test)

In [ ]:
data.head()

In [ ]:
data.drop(['id'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
def pre_process(s): #Remove Punctuations
    s = s.str.lower() #Lower all text
    s = s.str.replace(r'(?i)\brt\b', " ", regex = True)
    s = s.str.replace(' via '," ") 
    s = s.replace(r'@\w+', " ", regex = True)
    s = s.replace(r'http\S+', ' ', regex = True)
    s = s.replace(r'www.[^ ]+', ' ', regex = True)
    s = s.replace(r'[0-9]+', ' ', regex = True)
    s = s.replace(r'''[¬!"#$%&()*+,-./:;<=>?@[\]’^'_`\{|}~]''', ' ', regex = True)
    return s    
data['comment_text'] = pre_process(data.comment_text)

In [ ]:
data.head()

In [ ]:
#remove \n from whole dataframe
data = data.replace('\n',' ', regex=True)


In [ ]:
pd.options.display.max_colwidth =1000

In [ ]:
data.comment_text[2]

In [ ]:
#remove single character from the string
data.comment_text = data.comment_text.str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')


In [ ]:
data.comment_text[2]

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('english')
data['comment_text']  = data['comment_text'].apply([lambda x: [item for item in word_tokenize(x) if item not in stop_words]])


In [ ]:
data.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data['comment_text'] = data['comment_text'].apply(lambda x: [lemmatizer.lemmatize(item,'v') for item in x])

In [ ]:
data.head()

In [ ]:
from collections import Counter
a = data['comment_text']
a = [x for i in a for x in i]
top_20_train = pd.DataFrame(Counter(a).most_common(50), columns=['word', 'frequency'])
print(top_20_train)


In [ ]:
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join(x))

In [ ]:
data.head()

In [ ]:
!pip install ktrain

In [ ]:
import ktrain
from ktrain import text

In [ ]:
train_clean = data[:159571]
test_clean =  data[159571:]
test_clean.drop(['toxic','severe_toxic','obscene','threat','insult','identity_hate'], axis=1, inplace =True)

In [ ]:
train_clean.head()

In [ ]:
x = train_clean['comment_text'].to_list()
y = train_clean[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values.tolist()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y)

In [ ]:
label = train.columns.to_list()

In [ ]:
label.remove('id')
label.remove('comment_text')
label

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                          x_test=X_test, y_test=y_test,
                                          class_names=label,
                                          preprocess_mode='bert',
                                          maxlen=128)
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)


In [ ]:
learner.fit_onecycle(lr=2e-5,epochs=2)